In [24]:
library(tidyverse) #Collection of packages in the tidyverse (see https://www.tidyverse.org/)
library(gsubfn)


#  John Hopkins University data

# Sciensano data

## Mortality data

In [119]:
url_mortality_data = paste('https://epistat.sciensano.be/Data/COVID19BE_MORT.csv')
mortality_data = read.csv(url(url_mortality_data),stringsAsFactors = FALSE)

In [120]:
mortality_data = mortality_data %>% 
    mutate(date=DATE, location=REGION,value=DEATHS) %>%
    select(date,location,value) %>%
    group_by(date,location) %>%
    summarize(value=sum(value,na.rm=TRUE)) 

In [121]:
mortality_data_country = mortality_data %>% 
    group_by(date) %>% 
    summarize(value=sum(value,na.rm=TRUE)) %>%
    mutate(location="Belgium")

In [122]:
mortality_data = bind_rows(mortality_data,mortality_data_country) %>%
    arrange(date,location)

In [123]:
write.csv(file="data-truth/truth-incident-deaths-sciensano.csv", x=mortality_data, row.names=F)

In [124]:
truth = bind_rows(
  read_csv("data-truth/truth-incident-deaths-sciensano.csv") %>% 
    mutate(inc_cum = "inc", unit = "day"),
  
  read_csv("data-truth/truth-incident-deaths-sciensano.csv") %>% 
    dplyr::mutate(week = MMWRweek::MMWRweek(date)$MMWRweek) %>%
    dplyr::group_by(location,week) %>%
    dplyr::summarize(date = max(date),
                     value = sum(value, na.rm = TRUE)) %>%
    dplyr::filter(weekdays(date) == "Saturday") %>%
    dplyr::mutate(inc_cum = "inc", unit = "wk") %>%
    dplyr::select(-week)
)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  location = col_character(),
  value = col_double()
)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  location = col_character(),
  value = col_double()
)



In [125]:
head(truth)

date,location,value,inc_cum,unit
<date>,<chr>,<dbl>,<chr>,<chr>
2020-03-10,Belgium,1,inc,day
2020-03-10,Brussels,1,inc,day
2020-03-11,Belgium,3,inc,day
2020-03-11,Brussels,2,inc,day
2020-03-11,Flanders,1,inc,day
2020-03-12,Belgium,1,inc,day


# YYG

In [116]:
yyg_to_reich<-function(yyg_data, forecast_date, horizon) {
    
    yyg_data <- yyg_data %>% mutate(date=as.Date(date,"%Y-%m-%d"))
    
    predictions_yyg = yyg_data[yyg_data$date>forecast_date,]
    predictions_yyg = predictions_yyg[1:horizon,]
    
    ygg_reich_inc_death = data.frame(value=predictions_yyg[,"predicted_deaths_mean"], target_end_date=predictions_yyg$date)
    ygg_reich_inc_death = ygg_reich_inc_death %>% 
        mutate(forecast_date = forecast_date) %>%
        mutate(target = paste(1:horizon, " day ahead inc death",sep="")) %>%
        mutate(location = "Belgium") %>%
        mutate(type = "point") %>%
        mutate(quantile = NA)
    ygg_reich_inc_death = ygg_reich_inc_death[,c("forecast_date","target","target_end_date","location","type","quantile","value")]

    ygg_reich_inc_death_lower = data.frame(value=predictions_yyg[,"predicted_deaths_lower"], target_end_date=predictions_yyg$date)
    ygg_reich_inc_death_lower = ygg_reich_inc_death_lower %>% 
        mutate(forecast_date = forecast_date) %>%
        mutate(target = paste(1:horizon, " day ahead inc death",sep="")) %>%
        mutate(location = "Belgium") %>%
        mutate(type = "quantile") %>%
        mutate(quantile = 0.025)
    ygg_reich_inc_death_lower = ygg_reich_inc_death_lower[,c("forecast_date","target","target_end_date","location","type","quantile","value")]

    ygg_reich_inc_death_upper = data.frame(value=predictions_yyg[,"predicted_deaths_upper"], target_end_date=predictions_yyg$date)
    ygg_reich_inc_death_upper = ygg_reich_inc_death_upper %>% 
        mutate(forecast_date = forecast_date) %>%
        mutate(target = paste(1:horizon, " day ahead inc death",sep="")) %>%
        mutate(location = "Belgium") %>%
        mutate(type = "quantile") %>%
        mutate(quantile = 0.975)
    ygg_reich_inc_death_upper = ygg_reich_inc_death_upper[,c("forecast_date","target","target_end_date","location","type","quantile","value")]

    ygg_reich = bind_rows(ygg_reich_inc_death, ygg_reich_inc_death_lower, ygg_reich_inc_death_upper)
    
    ygg_reich
}

In [117]:
as.character(as.Date("2020-04-13","%Y-%m-%d")+(0:5)*7)

[1] "2020-04-13" "2020-04-20" "2020-04-27" "2020-05-04" "2020-05-11"
[6] "2020-05-18"

In [118]:
dir.create(file.path("data-processed/YYG-ParamSearch"), showWarnings = FALSE)

horizon = 28
forecast_dates = as.character(as.Date("2020-04-13","%Y-%m-%d")+(0:5)*7)

for (forecast_date in forecast_dates) {
    
    url_yyg_data = paste('https://raw.githubusercontent.com/youyanggu/covid19_projections/master/projections/',forecast_date,'/global/Belgium_ALL.csv',sep="")
    yyg_data = read.csv(url(url_yyg_data),stringsAsFactors = FALSE)
    
    ygg_reich = yyg_to_reich(yyg_data, forecast_date, horizon)
    
    write.csv(file=paste("data-processed/YYG-ParamSearch/",forecast_date,"-YYG-ParamSearch.csv",sep=""), x=ygg_reich, row.names=F) 
    
}